# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  1.84it/s]
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，它的卦象是由三个阳爻夹一个阴爻构成，象征着天、云、雷等现象。乾卦的含义非常丰富，它不仅代表了阳、力量、刚强、积极等含义，还代表了天、自然、运行、变化等含义。乾卦通常被解释为“天行健”，意味着天地自然界的运行是健壮有力的，它推动着一切事物的发展和变化。在人类的日常生活中，乾卦也象征着人们应该积极向前，勇往直前，迎接挑战，发挥自己的潜力和力量，以达到自己的目标。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它的卦象是由两个阳爻夹一个阴爻构成，象征着诉讼、争端、矛盾等纷争现象。讼卦的含义非常丰富，它不仅代表了诉讼、争端、矛盾等负面含义，还代表了和解、协调、平衡等积极含义。

在周易中，讼卦通常被解释为“天听之”，意味着天地自然界的运行是公正无私的，它听任事物的发展和变化，不偏袒任何一方。人类社会中的诉讼、争端、矛盾等纷争现象，也需要通过公正、公平、客观的分析和解决，来实现社会的和谐、稳定和健康发展。因此，讼卦也象征着人们应该倾听他人的声音，尊重他人的权利和利益，通过平等、协商、妥协等方式，来解决纷争和矛盾，实现社会的和谐、稳定和健康发展。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250727_190226"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250727_190226）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250727_190226）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。坎象征着险恶的水流，乾象征着天，两者相交，形成一种复杂的卦象。在这个卦象中，天与水相分离，预示着君子应当避免涉足险恶的水流，而是要依靠君子自身的德行来避免灾难。尽管初始吉祥，但最终警惕性极高的的风险将会出现。因此，君子应当谨慎行事，避免冒险，以确保安全。对于君子而言，应当遵循道德准则，行事光明磊落，避免矛盾和纠纷。讼卦的核心哲学是：避免险恶，依赖德行，谨慎行事。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250727_190226）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [16]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，也是八宫图中的第一个卦象。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等阳刚之象，代表着创造、领导、坚定、行动等含义。

乾卦的卦辞是“元、亨、利、贞”，其中，“元”表示天地之始，“亨”表示亨通顺利，“利”表示有利可图，“贞”表示正道。综合起来，乾卦的含义是：具有创造性和领导力的阳刚之象，能够顺利达成目标，并且遵循正道。

在八宫图中，乾卦位于北方，与事业、领导、自信、果敢等有关。乾卦旺相时，人们通常会具备强烈的领导欲望和创造力，能够积极行动并取得成功。相反，乾卦弱相时，可能会过于自信或独断专行，需要注意控制自己的情绪和行动。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？
乾卦是八卦之一，也是八宫图说、易经、易学中的重要组成部分。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等自然现象，以及君王、领导、权力等象征。乾卦的卦辞为“元、亨、利、贞”，意味着“正大光明、通顺利、永恒持久”。乾卦的五行属性为“木”，在八卦中属于阳卦，代表阳刚之气，具有领导、决断、创造等特性。

乾卦的含义非常丰富，可以引申为“天命”、“刚健”等。在易经中，乾卦的六爻可以代表不同的时间段，每一爻都有不同的含义和象征。例如，初爻代表事物刚刚开始，二爻代表事物已经发展了一段时间，三爻代表事物已经发展了一段时间并开始走向成熟，四爻代表事物已经达到成熟的状态，五爻代表事物已经达到了极盛状态，上爻则代表事物已经过了高潮，开始走向衰退。

乾卦的运用可以帮助人们了解事物的发展规律，预测未来的发展趋势，同时也可以指导人们如何应对不同的情况，使事物发展更加顺利。


In [17]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦，又称地水师卦、地水卦，是易经卦象中的一种。在易经的八卦中，地水师卦位于北方，属于阳卦。这个卦象象征着水在地上的运动，以及人们在地水上进行活动的状态。地水师卦具有以下几个特点：

1. 柔顺：地水师卦是由两个阴爻夹一个阳爻构成，具有柔顺的特性，表示水在地上的流动是柔顺的，人们在地水上活动时要保持柔顺，遵循自然规律。

2. 流动：地水师卦反映了水的流动特性。水流动不止，表示事物发展变化不断，人们需要适应这种变化，因变而通。

3. 适应：地水师卦教导人们要善于适应环境的变化。水在地上的流动需要适应地形、气候等环境因素，人们在生活中也要随着环境的变化而变化，做到顺应自然，与自然和谐共处。

4. 众寡：地水师卦中，阳爻代表领导、主动、积极，阴爻代表跟随、被动、消极。地水师卦告诉我们，在事物的发展过程中，领导者的地位和作用是至关重要的，要发挥领导者的作用，调动众人的积极性，共同推进事物的发展。

5. 刚中：地水师卦中，阳爻处于中位，表示事物发展变化的适度、中庸。人们处理事情时要保持适度的态度，避免偏激与极端，做到刚中。

地水师卦提醒人们在处理事物时要保持柔顺、流动、适应、积极、中庸的态度，以顺应自然、调整自身，达到事物发展的最佳状态。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？
地水师卦是易经中的一个卦象，由两个卦相组合而成：地卦和水分成两个卦，即地卦和风卦。地卦代表土地，象征柔顺、包容、承载；风卦代表风，象征自由、变化、进取。将两个卦相组合，形成地水师卦，象征土地承载水，寓意着柔顺包容、承载变化。

地水师卦的意义主要体现在以下几个方面：

1. 土地承载水，象征柔顺、包容、承载。土地是生命的源泉，水是生命的活力，土地承载水，寓意着柔顺、包容、承载变化。

2. 地水师卦还象征着变化和进取。风卦象征变化和进取，地卦象征柔顺和承载。组合起来，寓意着在保持柔顺、包容、承载变化的基础上，积极寻求变化和进取。

3. 在人际关系方面，地水师卦还象征着团结、合作和互助。土地和水分开，但它们又相互依存，共同构成了地水师卦。这象征着人们应该在团结合作的基础上，相互依存，共同发展。

总之，地水师卦是易经中的一种卦象，寓意着柔顺、包容、承载变化，

In [18]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
周易中的讼卦（象征诉讼）是由两个卦相组合而成的。一面是乾卦，代表天、刚、动、健等阳刚之象；另一面是坤卦，代表地、柔、静、顺等阴柔之象。讼卦表示天地之间的矛盾、冲突和争端。

在周易六十四卦中，讼卦位于互传卦序列的第三卦。互传卦是两个相邻的卦相互组合而成，如乾卦与坤卦、震卦与巽卦等。通过相互组合，形成一个新的卦象，表示事物之间的相互关联和影响。

讼卦的意义非常丰富，它既表示人们之间可能发生的争执和冲突，也暗示着通过协商、沟通和妥协，可以化解这些矛盾。从哲学角度来看，讼卦教导我们在面对冲突和争端时，要懂得尊重对方，保持克制和包容，以达到和谐共处的目的。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么？
周易中的讼卦是八卦之一，它的卦象是由两个离字叠加而成，离为火，火可以克金，因此讼卦象征着诉讼、争端和矛盾。

在周易中，讼卦位于兑卦之上，表示兑泽之水与离火相克，意味着诉讼和争端可能会因为水火相克而产生。同时，讼卦也预示着诉讼可能会带来一定的困扰和损失，但最终会得到公正的解决。

周易中的讼卦具有很强的哲学意义，它告诉我们，在面对矛盾和争端时，应该采取公正、公平的态度，通过沟通和协商来解决问题，而不是通过诉讼等极端手段。只有这样，才能真正实现和平、和谐的社会。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [19]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [20]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，也是六十四卦中的首卦。乾卦代表着天，它由两个阳爻夹一个阴爻构成，象征着阳性的能量和力量。乾卦的卦辞是：“元，亨，利，贞”，意味着乾卦是宇宙的运行规律，是通利的，有利于生存的，是正确的。

乾卦的卦象是由三个阳爻夹一个阴爻构成，象征着天，表现了宇宙的运行和力量。在《易经》中，阳爻代表阳性的能量和力量，阴爻代表阴性的能量和力量，二者相互配合，形成宇宙的运行规律。

乾卦的卦象是：初爻，一阴爻，二阳爻；二爻，二阴爻，一阳爻。

乾卦的卦辞：元，亨，利，贞。

白话文解释：元，表示万物；亨，表示通，通利；利，表示顺利，有利；贞，表示正。

乾卦的卦辞表示：万物的基础是天，天行通利，生存易于成功，行为正确。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的一种卦象，它是由六个阳爻（代表天、强、刚、锐）和一個阴爻（代表地、顺、和）组成的。乾卦象征着天，代表着刚强、锐利和正确，它意味着遵循道德原则，追求正确和正义，以及保持一种积极向上的态度。乾卦也象征着六种阳爻和一阴爻，意味着事物的发展和变化。在《易经》中，乾卦是八卦中第一卦，它代表着宇宙的运行和自然的法则，也象征着人类社会的法则和道德标准。乾卦的意义和象征意义取决于阴阳爻的组合，阴爻在下面，阳爻在上面，表示一种积极向上，刚强锐利的态度，也象征着宇宙的运行和自然的法则。


In [21]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦是《易经》中的卦象之一，属于师卦是师火卦下,师克卦上。师卦是由师的在上,师的居中,师的在下三个阳爻构成。师卦是反映师兵之卦,如兵之众,兵之师,兵之功,反映大众之功,火下有师,师下有师等。

师卦象征师兵,表明了师兵大众的作用,它是由下卦师卦上卦离组成的。离为火,火可以克木,木象征树,树可以生火,火功不可没。师卦告诉人们,师兵大众可以完成艰巨的任务,师卦也是表示记念师旅之卦。

师卦的阳爻在下,阴爻在上,表明了阴火记念阳木之功。师卦之记念火木功,表示记念师旅之功。

师卦之离火在上,表示记忆,记念,记取。师卦之师下有师,表示记住下面师旅之功。

师卦之象为火,象征火记念树之功,表明了记念师旅之功。师卦之吉凶取决于阴爻之数,阴爻越少则师卦越吉,阴爻越多则师卦越凶。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [22]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》中第6卦，由两个阴爻和三个阳爻组成。讼卦象征诉讼，表明阴爻与阳爻之间的矛盾和争端，但阴爻希望和平解决，阳爻则坚持要通过诉讼来解决。

讼卦的卦象是：上卦为阴，下卦为阳。阴爻在下，阳爻在上。

讼卦的卦辞：讼，大象：打仗。象征诉讼，困难险阻。患难在于险，困境在于德行不足，忧愁在于迷惑，敌进在于不断的威胁，亲顺在于和谐，险恶在于险阻，险斯至矣。

讼卦的卦辞：象曰：天与水违行，讼。君子以作谋，敬慎。

运势：讼卦象征诉讼，困境，忧愁，危险，恶运，险恶，德行不足。的能量很大，但缺德时则运术背离，财运远逝，好运尽失，诸事阻隔，需谨慎。

家宅：讼卦象征诉讼，也象征困境和忧愁，这些状况都会家居环境中表现出来。因此，对于家庭生活，应谨慎处理，避免纠纷，尽量求得和谐。

事业：讼卦象征诉讼，也象征困境和忧愁，这些状况都会工作中表现出来。因此，对于职业生涯，应谨慎处理，避免纠纷，尽量求得和谐。

婚恋：讼卦象征争端和矛盾，对于婚恋关系，应谨慎处理，避免纠纷，尽量求得和谐。

财运：讼卦象征诉讼，也象征困境和忧愁，这些状况都会财运中表现出来。因此，对于财运，应谨慎处理，避免纠纷，尽量求得和谐。

的健康：讼卦象征争端和矛盾，这些状况都会健康中表现出来。因此，对于健康，应谨慎处理，避免纠纷，尽量求得和谐。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
